In [2]:
from typing import Any, Dict, List, Optional
from langchain import BasePromptTemplate, PromptTemplate
from langchain.chains.base import Chain
from langchain.schema.language_model import BaseLanguageModel
from langchain.chains import LLMChain
from pydantic import Extra, Field
from langchain.callbacks.manager import (
    AsyncCallbackManagerForChainRun,
    CallbackManagerForChainRun,
)

In [3]:
from datetime import datetime
from DateConvert import Lunar
import json


class CalendarChain(Chain):
    """
    An example of a custom chain.
    """

    prompt: BasePromptTemplate
    """Prompt object to use."""
    llm: BaseLanguageModel
    output_key: str = "text"  #: :meta private:

    res_chain: Optional[LLMChain] = Field(default=None, exclude=True)

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True

    @property
    def input_keys(self) -> List[str]:
        """Will be whatever keys the prompt expects.

        :meta private:
        """
        return self.prompt.input_variables

    @property
    def output_keys(self) -> List[str]:
        """Will always return text key.

        :meta private:
        """
        return [self.output_key]

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        prompt_value = self.prompt.format_prompt(**inputs)
        if run_manager:
            run_manager.on_text(
                prompt_value.to_string(), color="green", end="\n", verbose=self.verbose
            )
        result = self.llm.generate_prompt(
            [prompt_value], callbacks=run_manager.get_child() if run_manager else None
        )
        if run_manager:
            run_manager.on_text(
                result.generations[0][0].text,
                color="yellow",
                end="\n",
                verbose=self.verbose,
            )
            date_json = json.loads(result.generations[0][0].text)
            res = ""
        for dstr in date_json:
            try:
                d = datetime.strptime(dstr, "%Y-%m-%d %H:%M")
                ld = Lunar(d)
                res = (
                    res
                    + f"{dstr}是{ld.ln_date_str()} {ld.sx_year()}年；生辰八字是{ld.gz_year()}年 {ld.gz_month()}月 {ld.gz_day()}日 {ld.gz_hour()}时"
                )
            except ValueError as e:
                d = datetime.strptime(dstr, "%Y-%m-%d %H:%M:%S")
                ld = Lunar(d)
                res = (
                    res
                    + f"{dstr}是{ld.ln_date_str()} {ld.sx_year()}年；生辰八字是{ld.gz_year()}年 {ld.gz_month()}月 {ld.gz_day()}日 {ld.gz_hour()}时"
                )
        return {self.output_key: res}

    async def _acall(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[AsyncCallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        prompt_value = self.prompt.format_prompt(**inputs)
        if run_manager:
            await run_manager.on_text(
                prompt_value.to_string(), color="green", end="\n", verbose=self.verbose
            )
        result = await self.llm.agenerate_prompt(
            [prompt_value], callbacks=run_manager.get_child() if run_manager else None
        )
        if run_manager:
            await run_manager.on_text(
                result.generations[0][0].text,
                color="yellow",
                end="\n",
                verbose=self.verbose,
            )
        date_json = json.loads(result.generations[0][0].text)
        res = ""
        for dstr in date_json:
            try:
                d = datetime.strptime(dstr, "%Y-%m-%d %H:%M")
                ld = Lunar(d)
                res = (
                    res
                    + f"{dstr}是{ld.ln_date_str()} {ld.sx_year()}年；生辰八字是{ld.gz_year()}年 {ld.gz_month()}月 {ld.gz_day()}日 {ld.gz_hour()}时"
                )
            except ValueError as e:
                d = datetime.strptime(dstr, "%Y-%m-%d %H:%M:%S")
                ld = Lunar(d)
                res = (
                    res
                    + f"{dstr}是{ld.ln_date_str()} {ld.sx_year()}年；生辰八字是{ld.gz_year()}年 {ld.gz_month()}月 {ld.gz_day()}日 {ld.gz_hour()}时"
                )

        return {self.output_key: res}

    @property
    def _chain_type(self) -> str:
        return "calendar_chain"

    @classmethod
    def from_llm(
        cls,
        llm: BaseLanguageModel,
        **kwargs: Any,
    ) -> Chain:
        template = """说明
---------------------
- 下面的内容包含一个或者多个日期。

内容
---------------------
{question}

要求
---------------------
- 将内容中的日期提取出来生成一个json数组字符串，数组的元素是每一个公历日期。
- 如果上面的问题不包含任何日期数据，请生成一个空的json数组字符串。
- 只生成json字符串即可，不需要生成其他任何非json格式的字符。
- 日期的格式为%Y-%m-%d %H:%M

Answer
---------------------"""
        prompt = PromptTemplate.from_template(template=template)
        return cls(llm=llm, prompt=prompt, **kwargs)

/tmp/ipykernel_8781/2650707974.py:21: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.3/migration/
  extra = Extra.forbid


In [4]:
import pandas as pd


def getStrokeCount(target) -> int:
    df = pd.read_csv("chinese_unicode_table.csv")
    matches = df[df.iloc[:, 0] == target]  # df[0] 表示第1列，df[6] 表示第7列
    re = matches.iloc[0, 6]
    return int(re) % 10

In [5]:
class FiveElementsTheoryOfNameChain(Chain):
    """
    An example of a custom chain.
    """

    prompt: BasePromptTemplate
    """Prompt object to use."""
    llm: BaseLanguageModel
    output_key: str = "text"  #: :meta private:

    res_chain: Optional[LLMChain] = Field(default=None, exclude=True)

    five_elements = ["水", "木", "木", "火", "火", "土", "土", "金", "金", "水"]

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True

    @property
    def input_keys(self) -> List[str]:
        """Will be whatever keys the prompt expects.

        :meta private:
        """
        return self.prompt.input_variables

    @property
    def output_keys(self) -> List[str]:
        """Will always return text key.

        :meta private:
        """
        return [self.output_key]

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        prompt_value = self.prompt.format_prompt(**inputs)
        if run_manager:
            run_manager.on_text(
                prompt_value.to_string(), color="green", end="\n", verbose=self.verbose
            )
        result = self.llm.agenerate_prompt(
            [prompt_value], callbacks=run_manager.get_child() if run_manager else None
        )
        name = result.generations[0][0].text.strip()
        origin_name = [None] * len(name)
        if name != "None":
            origin_name = list(name)
        else:
            return {self.output_key: "I can't analysis your name."}
        if len(origin_name) < 2 or len(origin_name) > 4:
            return {self.output_key: "I can't analysis your name."}
        stroke_count = [None] * 4
        if len(origin_name) == 2:
            stroke_count = [
                1,
                getStrokeCount(origin_name[0]),
                1,
                getStrokeCount(origin_name[1]),
            ]
        if len(origin_name) == 3:
            stroke_count = [
                1,
                getStrokeCount(origin_name[0]),
                getStrokeCount(origin_name[1]),
                getStrokeCount(origin_name[2]),
            ]
        if len(origin_name) == 4:
            stroke_count = [
                getStrokeCount(origin_name[0]),
                getStrokeCount(origin_name[1]),
                getStrokeCount(origin_name[2]),
                getStrokeCount(origin_name[3]),
            ]
        five_grid_num = [
            x % 10
            for x in [
                stroke_count[0] + stroke_count[1],
                stroke_count[2] + stroke_count[3],
                stroke_count[1] + stroke_count[2],
                stroke_count[0] + stroke_count[1] + stroke_count[2] + stroke_count[3],
                stroke_count[0] + stroke_count[3],
            ]
        ]
        res = f"名字'{name}'的五格分别是：天格{self.five_elements[five_grid_num[0]]},地格{self.five_elements[five_grid_num[1]]},人格{self.five_elements[five_grid_num[2]]},总格{self.five_elements[five_grid_num[3]]},外格{self.five_elements[five_grid_num[4]]}"
        return {self.output_key: res}

    async def _acall(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[AsyncCallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        prompt_value = self.prompt.format_prompt(**inputs)
        if run_manager:
            await run_manager.on_text(
                prompt_value.to_string(), color="green", end="\n", verbose=self.verbose
            )
        result = await self.llm.agenerate_prompt(
            [prompt_value], callbacks=run_manager.get_child() if run_manager else None
        )
        name = result.generations[0][0].text.strip()
        origin_name = [None] * len(name)
        if name != "None":
            origin_name = list(name)
        else:
            return {self.output_key: "I can't analysis your name."}
        if len(origin_name) < 2 or len(origin_name) > 4:
            return {self.output_key: "I can't analysis your name."}
        stroke_count = [None] * 4
        if len(origin_name) == 2:
            stroke_count = [
                1,
                getStrokeCount(origin_name[0]),
                1,
                getStrokeCount(origin_name[1]),
            ]
        if len(origin_name) == 3:
            stroke_count = [
                1,
                getStrokeCount(origin_name[0]),
                getStrokeCount(origin_name[1]),
                getStrokeCount(origin_name[2]),
            ]
        if len(origin_name) == 4:
            stroke_count = [
                getStrokeCount(origin_name[0]),
                getStrokeCount(origin_name[1]),
                getStrokeCount(origin_name[2]),
                getStrokeCount(origin_name[3]),
            ]
        five_grid_num = [
            x % 10
            for x in [
                stroke_count[0] + stroke_count[1],
                stroke_count[2] + stroke_count[3],
                stroke_count[1] + stroke_count[2],
                stroke_count[0] + stroke_count[1] + stroke_count[2] + stroke_count[3],
                stroke_count[0] + stroke_count[3],
            ]
        ]
        res = f"名字'{name}'的五格分别是：天格{self.five_elements[five_grid_num[0]]},地格{self.five_elements[five_grid_num[1]]},人格{self.five_elements[five_grid_num[2]]},总格{self.five_elements[five_grid_num[3]]},外格{self.five_elements[five_grid_num[4]]}"
        return {self.output_key: res}

    @property
    def _chain_type(self) -> str:
        return "calendar_chain"

    @classmethod
    def from_llm(
        cls,
        llm: BaseLanguageModel,
        **kwargs: Any,
    ) -> Chain:
        template = """说明
---------------------
- 下面文本内容中都是中文

文本内容
---------------------
{content}

要求
---------------------
- 请把文本内容中包含的中文名字或者名词提取出来。
- 如果文本内容中不包含任何中文名字，请回答None。

Answer
---------------------"""
        prompt = PromptTemplate.from_template(template=template)
        return cls(llm=llm, prompt=prompt, **kwargs)

/tmp/ipykernel_8781/3973105116.py:18: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.3/migration/
  extra = Extra.forbid


```
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain import OpenAI

load_dotenv(dotenv_path="env")
chat_gpt35 = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613", verbose=True)
gpt35_1 = OpenAI(temperature=0.1, verbose=True)
chain = FiveElementsTheoryOfNameChain.from_llm(llm=gpt35_1, verbose=True)
content = input("Your prompt: ")
res = await chain.arun(content)
print(res)
```

In [6]:
from typing import Any
from langchain.callbacks.base import AsyncCallbackHandler


class StreamingLLMCallbackHandler(AsyncCallbackHandler):
    """Callback handler for streaming LLM responses."""

    # def __init__(self):
    # self.websocket = websocket

    # async def on_llm_new_token(self, token: str, **kwargs: Any) -> None:
    #     # resp = ChatResponse(sender="bot", message=token, type="stream")
    #     # await self.websocket.send_json(resp.dict())
    #     print(f"TOKEN: {token}")

In [7]:
from datetime import datetime
import time


class MyCalendar:
    @classmethod
    def now(cls, question: str, **kwargs: any) -> str:
        return f"It is {datetime.now()} {time.tzname[0]} now."

    @classmethod
    async def anow(cls, question: str, **kwargs: any) -> str:
        return f"It is {datetime.now()} {time.tzname[0]} now."

In [8]:
from langchain import OpenAI


class TradingViewReasearchReportChain(Chain):
    """
    An example of a custom chain.
    """

    prompt: BasePromptTemplate
    """Prompt object to use."""
    llm: BaseLanguageModel
    output_key: str = "text"  #: :meta private:

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True

    @property
    def input_keys(self) -> List[str]:
        """Will be whatever keys the prompt expects.

        :meta private:
        """
        return self.prompt.input_variables

    @property
    def output_keys(self) -> List[str]:
        """Will always return text key.

        :meta private:
        """
        return [self.output_key]

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        # Your custom chain logic goes here
        # This is just an example that mimics LLMChain
        prompt_value = self.prompt.format_prompt(**inputs)
        if run_manager:
            run_manager.on_text(
                prompt_value.to_string(),
                color="green",
                end="\n",
                verbose=self.verbose,
            )
        # Whenever you call a language model, or another chain, you should pass
        # a callback manager to it. This allows the inner run to be tracked by
        # any callbacks that are registered on the outer run.
        # You can always obtain a callback manager for this by calling
        # `run_manager.get_child()` as shown below.
        response = self.llm.generate_prompt(
            [prompt_value], callbacks=run_manager.get_child() if run_manager else None
        )

        # If you want to log something about this run, you can do so by calling
        # methods on the `run_manager`, as shown below. This will trigger any
        # callbacks that are registered for that event.
        # answer=self.answer_chain.run(question=inputs['user_input'],context=res)
        if run_manager:
            run_manager.on_text(
                response.generations[0][0].text,
                color="yellow",
                end="\n",
                verbose=self.verbose,
            )
        return {self.output_key: response.generations[0][0].text}

    async def _acall(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[AsyncCallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        # Your custom chain logic goes here
        # This is just an example that mimics LLMChain
        prompt_value = self.prompt.format_prompt(**inputs)
        if run_manager:
            await run_manager.on_text(
                prompt_value.to_string(),
                color="green",
                end="\n",
                verbose=self.verbose,
            )
        # Whenever you call a language model, or another chain, you should pass
        # a callback manager to it. This allows the inner run to be tracked by
        # any callbacks that are registered on the outer run.
        # You can always obtain a callback manager for this by calling
        # `run_manager.get_child()` as shown below.
        response = await self.llm.agenerate_prompt(
            [prompt_value], callbacks=run_manager.get_child() if run_manager else None
        )

        # If you want to log something about this run, you can do so by calling
        # methods on the `run_manager`, as shown below. This will trigger any
        # callbacks that are registered for that event.
        if run_manager:
            await run_manager.on_text(
                response.generations[0][0].text,
                color="yellow",
                end="\n",
                verbose=self.verbose,
            )
        return {self.output_key: response.generations[0][0].text}

    @property
    def _chain_type(self) -> str:
        return "indicators_questions_chain"

    @classmethod
    def from_llm(
        cls,
        llm,
        **kwargs: Any,
    ) -> Chain:
        PROMPT_TEMPLATE = """We have an unofficial python API wrapper to retrieve technical analysis from TradingView.
Retrieving the analysis:
- summary: Technical analysis (based on both oscillators and moving averages).
```
# Example
{{'RECOMMENDATION': 'BUY', 'BUY': 12, 'SELL': 7, 'NEUTRAL': 9}}
```
- oscillators: Technical analysis (based on oscillators).
```
# Example
{{'RECOMMENDATION': 'BUY', 'BUY': 2, 'SELL': 1, 'NEUTRAL': 8, 'COMPUTE': {{'RSI': 'NEUTRAL', 'STOCH.K': 'NEUTRAL', 'CCI': 'NEUTRAL', 'ADX': 'NEUTRAL', 'AO': 'NEUTRAL', 'Mom': 'BUY', 'MACD': 'SELL', 'Stoch.RSI': 'NEUTRAL', 'W%R': 'NEUTRAL', 'BBP': 'BUY', 'UO': 'NEUTRAL'}}}}
```
- moving_averages: Technical analysis (based on moving averages).
```
# Example
{{'RECOMMENDATION': 'BUY', 'BUY': 9, 'SELL': 5, 'NEUTRAL': 1, 'COMPUTE': {{'EMA10': 'SELL', 'SMA10': 'SELL', 'EMA20': 'SELL', 'SMA20': 'SELL', 'EMA30': 'BUY', 'SMA30': 'BUY', 'EMA50': 'BUY', 'SMA50': 'BUY', 'EMA100': 'BUY', 'SMA100': 'BUY', 'EMA200': 'BUY', 'SMA200': 'BUY', 'Ichimoku': 'NEUTRAL', 'VWMA': 'SELL', 'HullMA': 'BUY'}}}}
```
- indicators: Technical indicators.
```
# Example
{{'Recommend.Other': 0, 'Recommend.All': 0.26666667, 'Recommend.MA': 0.53333333, 'RSI': 60.28037412, 'RSI[1]': 58.58364778, 'Stoch.K': 73.80404453, 'Stoch.D': 79.64297643, 'Stoch.K[1]': 78.88160227, 'Stoch.D[1]': 85.97647064, 'CCI20': 46.58442886, 'CCI20[1]': 34.57058796, 'ADX': 35.78754863, 'ADX+DI': 23.16948389, 'ADX-DI': 13.82449817, 'ADX+DI[1]': 24.15991909, 'ADX-DI[1]': 13.87125505, 'AO': 6675.72158824, 'AO[1]': 7283.92420588, 'Mom': 1532.6, 'Mom[1]': 108.29, 'MACD.macd': 2444.73734978, 'MACD.signal': 2606.00138275, 'Rec.Stoch.RSI': 0, 'Stoch.RSI.K': 18.53740187, 'Rec.WR': 0, 'W.R': -26.05634845, 'Rec.BBPower': 0, 'BBPower': 295.52055898, 'Rec.UO': 0, 'UO': 55.68311917, 'close': 45326.97, 'EMA5': 45600.06414333, 'SMA5': 45995.592, 'EMA10': 45223.22433151, 'SMA10': 45952.635, 'EMA20': 43451.52018338, 'SMA20': 43609.214, 'EMA30': 41908.5944052, 'SMA30': 40880.391, 'EMA50': 40352.10222373, 'SMA50': 37819.3566, 'EMA100': 40356.09177879, 'SMA100': 38009.7808, 'EMA200': 39466.50411569, 'SMA200': 45551.36135, 'Rec.Ichimoku': 0, 'Ichimoku.BLine': 40772.57, 'Rec.VWMA': 1, 'VWMA': 43471.81729377, 'Rec.HullMA9': -1, 'HullMA9': 45470.37107407, 'Pivot.M.Classic.S3': 11389.27666667, 'Pivot.M.Classic.S2': 24559.27666667, 'Pivot.M.Classic.S1': 33010.55333333, 'Pivot.M.Classic.Middle': 37729.27666667, 'Pivot.M.Classic.R1': 46180.55333333, 'Pivot.M.Classic.R2': 50899.27666667, 'Pivot.M.Classic.R3': 64069.27666667, 'Pivot.M.Fibonacci.S3': 24559.27666667, 'Pivot.M.Fibonacci.S2': 29590.21666667, 'Pivot.M.Fibonacci.S1': 32698.33666667, 'Pivot.M.Fibonacci.Middle': 37729.27666667, 'Pivot.M.Fibonacci.R1': 42760.21666667, 'Pivot.M.Fibonacci.R2': 45868.33666667, 'Pivot.M.Fibonacci.R3': 50899.27666667, 'Pivot.M.Camarilla.S3': 37840.08, 'Pivot.M.Camarilla.S2': 39047.33, 'Pivot.M.Camarilla.S1': 40254.58, 'Pivot.M.Camarilla.Middle': 37729.27666667, 'Pivot.M.Camarilla.R1': 42669.08, 'Pivot.M.Camarilla.R2': 43876.33, 'Pivot.M.Camarilla.R3': 45083.58, 'Pivot.M.Woodie.S3': 21706.84, 'Pivot.M.Woodie.S2': 25492.42, 'Pivot.M.Woodie.S1': 34876.84, 'Pivot.M.Woodie.Middle': 38662.42, 'Pivot.M.Woodie.R1': 48046.84, 'Pivot.M.Woodie.R2': 51832.42, 'Pivot.M.Woodie.R3': 61216.84, 'Pivot.M.Demark.S1': 35369.915, 'Pivot.M.Demark.Middle': 38908.9575, 'Pivot.M.Demark.R1': 48539.915, 'open': 44695.95, 'P.SAR': 48068.64, 'BB.lower': 37961.23510877, 'BB.upper': 49257.19289123, 'AO[2]': 7524.31223529, 'volume': 32744.424503, 'change': 1.44612354, 'low': 44203.28, 'high': 45560}}
```

We got the analysis data of {symbol} from python-tradingview-ta as following:
summary:{summary}
oscillators:{oscillators}
moving_averages:{moving_averages}
indicators:{indicators}
"""
        PROMPT_TEMPLATE = (
            PROMPT_TEMPLATE
            + """\nPlease generate the analysis results by analyzing data the above, and provide the market trend. If the strength of the signal is represented by a score from 1 to 10, where a higher score indicates a stronger signal, please give a score to the strength of the signal in the end.
Your generation:
"""
        )

        prompt = PromptTemplate.from_template(PROMPT_TEMPLATE)
        return cls(llm=llm, prompt=prompt, **kwargs)

/tmp/ipykernel_8781/2375183569.py:17: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.3/migration/
  extra = Extra.forbid


In [9]:
pip install tradingview-ta


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
from tradingview_ta import TA_Handler, Interval, Exchange


class TradingviewWrapper:
    llm: BaseLanguageModel

    def __init__(cls, llm):
        cls.llm = llm

    def buySellSignal(cls, symbol: str, **kwargs: any) -> str:
        btc_usdt = TA_Handler(
            symbol=f"{symbol}USDT",
            screener="crypto",
            exchange="GATEIO",
            interval=Interval.INTERVAL_1_DAY,
        )
        summary = btc_usdt.get_analysis().summary
        oscillators = btc_usdt.get_analysis().oscillators
        moving_averages = btc_usdt.get_analysis().moving_averages
        indicators = btc_usdt.get_analysis().indicators
        # return "\n".join([f"Summary:{summary}",f"Oscillators:{oscillators}",f"Moving Averages:{moving_averages}",f"Indicators:{indicators}"])
        tradingview_chain = TradingViewReasearchReportChain.from_llm(
            llm=cls.llm, verbose=True
        )
        return tradingview_chain.run(
            symbol=f"{symbol}/usdt",
            summary=summary,
            oscillators=oscillators,
            moving_averages=moving_averages,
            indicators=indicators,
        )

    async def abuySellSignal(cls, symbol: str, **kwargs: any) -> str:
        btc_usdt = TA_Handler(
            symbol=f"{symbol}USDT",
            screener="crypto",
            exchange="GATEIO",
            interval=Interval.INTERVAL_1_DAY,
        )
        summary = btc_usdt.get_analysis().summary
        oscillators = btc_usdt.get_analysis().oscillators
        moving_averages = btc_usdt.get_analysis().moving_averages
        indicators = btc_usdt.get_analysis().indicators
        # return "\n".join([f"Summary:{summary}",f"Oscillators:{oscillators}",f"Moving Averages:{moving_averages}",f"Indicators:{indicators}"])
        tradingview_chain = TradingViewReasearchReportChain.from_llm(
            llm=cls.llm, verbose=True
        )
        return await tradingview_chain.arun(
            symbol=f"{symbol}/usdt",
            summary=summary,
            oscillators=oscillators,
            moving_averages=moving_averages,
            indicators=indicators,
        )

In [11]:
import requests
import json


def getPlaceId(address: str, api_key: str) -> str:
    endpoint_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
    params = {"input": address, "inputtype": "textquery", "key": api_key}

    res = requests.get(endpoint_url, params=params)
    result = json.loads(res.content)

    if result["status"] != "OK":
        raise Exception(result["status"])
    return result["candidates"][0]["place_id"]


def geocode(placeid: str, api_key: str) -> (float, float):
    endpoint_url = "https://maps.googleapis.com/maps/api/place/details/json"
    params = {"place_id": placeid, "key": api_key}

    res = requests.get(endpoint_url, params=params)
    result = json.loads(res.content)

    if result["status"] != "OK":
        raise Exception(result["status"])
    lat = result["result"]["geometry"]["location"]["lat"]
    lng = result["result"]["geometry"]["location"]["lng"]

    return (lat, lng)


def find_places(radius_meters: str, lat: float, lng: float, api_key: str) -> str:
    endpoint_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": f"{lat},{lng}",  # "lat,long" format
        "radius": radius_meters,
        "key": api_key,
        "language": "zh",
    }

    res = requests.get(endpoint_url, params=params)
    results = json.loads(res.content)
    if results["status"] != "OK":
        raise Exception(results["status"])
    places = results["results"]
    if len(places) > 0:
        content_strs = [
            f"{p['name']:45}lat:{p['geometry']['location']['lat']},lng:{p['geometry']['location']['lng']}{'':5}|{'':5}types:{','.join(p['types'])}"
            for p in places
        ]
        return "\n".join(content_strs)
    else:
        return ""

In [12]:
import os


class NearbySearchChain(Chain):
    """
    An example of a custom chain.
    """

    prompt: BasePromptTemplate
    res_prompt: BasePromptTemplate
    """Prompt object to use."""
    llm: BaseLanguageModel
    output_key: str = "text"  #: :meta private:
    API_KEY: str

    res_chain: Optional[LLMChain] = Field(default=None, exclude=True)

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True

    @property
    def input_keys(self) -> List[str]:
        """Will be whatever keys the prompt expects.

        :meta private:
        """
        return self.prompt.input_variables

    @property
    def output_keys(self) -> List[str]:
        """Will always return text key.

        :meta private:
        """
        return [self.output_key]

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        prompt_value = self.prompt.format_prompt(**inputs)
        if run_manager:
            run_manager.on_text(
                prompt_value.to_string(), color="green", end="\n", verbose=self.verbose
            )
        result = self.llm.generate_prompt(
            prompts=[prompt_value],
            callbacks=run_manager.get_child() if run_manager else None,
        )
        address = result.generations[0][0].text
        if run_manager:
            run_manager.on_text(address, color="yellow", end="\n", verbose=self.verbose)
        placeid = getPlaceId(address=address, api_key=self.API_KEY)
        (lat, lng) = geocode(placeid=placeid, api_key=self.API_KEY)
        places = find_places(radius_meters=5000, lat=lat, lng=lng, api_key=self.API_KEY)
        res_prompt_value = self.res_prompt.format_prompt(
            address=address, lat=lat, lng=lng, nearby=places
        )
        if run_manager:
            run_manager.on_text(
                res_prompt_value.to_string(),
                color="green",
                end="\n",
                verbose=self.verbose,
            )
        result = self.llm.generate_prompt(
            prompts=[res_prompt_value],
            callbacks=run_manager.get_child() if run_manager else None,
        )
        if run_manager:
            run_manager.on_text(
                result.generations[0][0].text,
                color="green",
                end="\n",
                verbose=self.verbose,
            )
        return {self.output_key: res_prompt_value.to_string()}

    async def _acall(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[AsyncCallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        prompt_value = self.prompt.format_prompt(**inputs)
        if run_manager:
            await run_manager.on_text(
                prompt_value.to_string(), color="green", end="\n", verbose=self.verbose
            )
        result = await self.llm.agenerate_prompt(
            prompts=[prompt_value],
            callbacks=run_manager.get_child() if run_manager else None,
        )
        address = result.generations[0][0].text
        if run_manager:
            await run_manager.on_text(
                address, color="yellow", end="\n", verbose=self.verbose
            )
        placeid = getPlaceId(address=address, api_key=self.API_KEY)
        (lat, lng) = geocode(placeid=placeid, api_key=self.API_KEY)
        places = find_places(
            radius_meters=10000, lat=lat, lng=lng, api_key=self.API_KEY
        )
        res_prompt_value = self.res_prompt.format_prompt(
            address=address, lat=lat, lng=lng, nearby=places
        )
        if run_manager:
            await run_manager.on_text(
                res_prompt_value.to_string(),
                color="green",
                end="\n",
                verbose=self.verbose,
            )
        result = await self.llm.agenerate_prompt(
            prompts=[res_prompt_value],
            callbacks=run_manager.get_child() if run_manager else None,
        )
        if run_manager:
            await run_manager.on_text(
                result.generations[0][0].text,
                color="green",
                end="\n",
                verbose=self.verbose,
            )
        return {self.output_key: res_prompt_value.to_string()}

    @property
    def _chain_type(self) -> str:
        return "nearby_search_chain"

    @classmethod
    def from_llm(
        cls,
        llm: BaseLanguageModel,
        **kwargs: Any,
    ) -> Chain:
        template = """说明
---------------------
- 请把下面问题中包含的地址提取出来

问题
---------------------
{question}

上面问题包含的地址
---------------------"""
        prompt = PromptTemplate.from_template(template=template)
        res_template = """中心地址
---------------------
{address}
Location:{lat},{lng}

地址附近
---------------------
{nearby}

要求
---------------------
- 根据“地址附近”的地点数据生成一段文字。
- 要求包含所有附近地点说明
- 要求说明附近地点与中心地址的距离。
- 要求说明附近地点在中心地址的什么方向
- 要求对附近地点做简要介绍
- 并总结附近地点的种类、

Answer
---------------------"""
        res_prompt = PromptTemplate.from_template(template=res_template)
        API_KEY = os.getenv("PLACES_API_KEY")
        return cls(
            llm=llm, prompt=prompt, res_prompt=res_prompt, API_KEY=API_KEY, **kwargs
        )

/tmp/ipykernel_8781/1043329439.py:21: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.3/migration/
  extra = Extra.forbid


In [13]:
from langchain.utilities.dalle_image_generator import DallEAPIWrapper
from pydantic import BaseModel


class ImageGenChain(Chain):
    """
    An example of a custom chain.
    """

    prompt: BasePromptTemplate
    """Prompt object to use."""
    llm: BaseLanguageModel
    img_model: DallEAPIWrapper
    output_key: str = "text"  #: :meta private:

    @staticmethod
    def result_prefix() -> str:
        return "IMG_URL:"

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True

    @property
    def input_keys(self) -> List[str]:
        """Will be whatever keys the prompt expects.

        :meta private:
        """
        return self.prompt.input_variables

    @property
    def output_keys(self) -> List[str]:
        """Will always return text key.

        :meta private:
        """
        return [self.output_key]

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        prompt_value = self.prompt.format_prompt(**inputs)
        if run_manager:
            run_manager.on_text(
                prompt_value.to_string(), color="green", end="\n", verbose=self.verbose
            )
        result = self.llm.generate_prompt(
            prompts=[prompt_value],
            callbacks=run_manager.get_child() if run_manager else None,
        )
        image_description = result.generations[0][0].text
        if run_manager:
            run_manager.on_text(
                image_description, color="yellow", end="\n", verbose=self.verbose
            )

        image_url = self.img_model.run(image_description)
        return {self.output_key: self.result_prefix() + image_url}

    async def _acall(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[AsyncCallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        prompt_value = self.prompt.format_prompt(**inputs)
        if run_manager:
            await run_manager.on_text(
                prompt_value.to_string(), color="green", end="\n", verbose=self.verbose
            )
        result = await self.llm.agenerate_prompt(
            prompts=[prompt_value],
            callbacks=run_manager.get_child() if run_manager else None,
        )
        image_description = result.generations[0][0].text
        if run_manager:
            await run_manager.on_text(
                image_description, color="yellow", end="\n", verbose=self.verbose
            )
        image_url = self.img_model.run(image_description)
        return {self.output_key: self.result_prefix() + image_url}

    @property
    def _chain_type(self) -> str:
        return "image_gen_chain"

    @classmethod
    def from_llm(
        cls,
        llm: BaseLanguageModel,
        **kwargs: Any,
    ) -> Chain:
        prompt = PromptTemplate(
            input_variables=["image_desc"],
            template="Generate a detailed prompt to generate an image based on the following description: {image_desc}",
        )
        img_model = DallEAPIWrapper(size="512x512")
        return cls(llm=llm, prompt=prompt, img_model=img_model, **kwargs)

/tmp/ipykernel_8781/4085041240.py:23: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.3/migration/
  extra = Extra.forbid


In [14]:
import json
import requests

import base64
import itertools
import re
from pathlib import Path

from pydantic import BaseModel
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool
from typing import List, Type

from langchain import BasePromptTemplate, PromptTemplate


class FileInfo(BaseModel):
    source_path: str
    description: str
    target_path: str


def head_file(path: str, n: int) -> List[str]:
    try:
        with open(path, "r") as f:
            return [str(line) for line in itertools.islice(f, n)]
    except Exception:
        return []


# import markdown
# from markdown.extensions.codehilite import CodeHiliteExtension
def strip_markdown_code(md_string: str) -> str:
    # stripped_string = re.sub(r"^`{1,3}.*?\n", "", md_string, flags=re.DOTALL)
    # stripped_string = re.sub(r"`{1,3}$", "", stripped_string)
    # return stripped_string
    # md = markdown.Markdown(md_string,extensions=[CodeHiliteExtension()])

    # # 转换markdown为html
    # html = md.convert(md_string)

    # # 对于每一个代码块，查看它是否是Python代码
    # for codeblock in md.parser.blockprocessors.get('codehilite').code_blocks:
    # 	if codeblock[0] == 'python':
    # 		return codeblock[1]
    code_blocks = re.findall(r"```python\r?\n(.*?)\r?\n```", md_string, re.S)
    return code_blocks[0]


def file_to_base64(path: str) -> str:
    with open(path, "rb") as f:
        return base64.b64encode(f.read()).decode()


def add_file(
    files, source_path: str, target_path: str, description: str
) -> Dict[str, FileInfo]:
    if files is None:
        files: Dict[str, FileInfo] = {}
    if target_path in files:
        raise ValueError("target_path already exists")
    if not Path(source_path).exists():
        raise ValueError("source_path does not exist")
    files[target_path] = FileInfo(
        target_path=target_path,
        source_path=source_path,
        description=description,
    )
    return files


def make_input_files(file_info_arr: any) -> List[dict]:
    files = []
    for file_info in file_info_arr:
        files.append(
            {
                "pathname": file_info["target_path"],
                "contentsBasesixtyfour": file_to_base64(file_info["source_path"]),
            }
        )
    return files

def file_description(files:any) -> str:
        if len(files) == 0:
            return ""
        lines = ["The following files available in the evaluation environment:"]
        for file_info in files:
            peek_content = head_file(file_info["source_path"], 4)
            lines.append(
                f"- path: `{file_info['target_path']}` \n first four lines: {peek_content}"
                f" \n description: `{file_info['description']}`"
            )
        return "\n".join(lines)


class CodeGenToolArguments(BaseModel):
    question: str = Field(
        ...,
        example="What is the first line of the csv file?",
        description=("The question to solved."),
    )
    file_info_arr_json: str = Field(
        ...,
        example='["source_path":"/home/user/1.txt","description":"It is a file for test.","target_path":"1.txt"]',
        description=(
            "An array string in JSON format. "
            "Each element represents the data of a file, including source_path, target_path and description. "
        ),
    )
    
class CodeReGenToolArguments(BaseModel):
    
    wrong_code: str = Field(
        ...,
        example="print('Hello world!')",
        description=(
            "The pure python script to be evaluated. "
            "The contents will be in main.py. "
            "It should not be in markdown format."
            "The source code that gave the error last time it was run."
        ),
    )
    stderr: str = Field(
        ...,
        example='Traceback (most recent call last):\n  File "/tmp/project/main.py", line 8, in <module>\n    reader = PyPDF2.PdfFileReader(file)\n  File "/venv/lib/python3.10/site-packages/PyPDF2/_reader.py", line 1974, in __init__\n    deprecation_with_replacement("PdfFileReader", "PdfReader", "3.0.0")\n  File "/venv/lib/python3.10/site-packages/PyPDF2/_utils.py", line 369, in deprecation_with_replacement\n    deprecation(DEPR_MSG_HAPPENED.format(old_name, removed_in, new_name))\n  File "/venv/lib/python3.10/site-packages/PyPDF2/_utils.py", line 351, in deprecation\n    raise DeprecationError(msg)\nPyPDF2.errors.DeprecationError: PdfFileReader is deprecated and was removed in PyPDF2 3.0.0. Use PdfReader instead.\n',
        description=(
            "Information returned when wrong_code Python code executes errors."
            "If the stderr parameter cannot be determined, assign it to an empty string."
        ),
    )
    
class PythonCodeGeneratorTool(BaseTool):
    name = "PythonCodeGeneratorTool"
    args_schema: Type[BaseModel] = CodeGenToolArguments
    description = """useful for when you can't answer the question directly, and need to generate python code."""

    llmChain: LLMChain

    def _run(self, question: str, file_info_arr_json: str) -> str:
        res = self.llmChain.run(
            question=question, file_info="Files Uploaded in JSON:" + file_info_arr_json
        )
        return res

    async def _arun(self, question: str, file_info_arr_json: str) -> str:
        res = await self.llmChain.arun(
            question=question, file_info="Files Uploaded in JSON:" + file_info_arr_json
        )
        return res

    @classmethod
    def from_llm(
        cls,
        llm: BaseLanguageModel,
        **kwargs: Any,
    ) -> BaseTool:
        prompt = PromptTemplate.from_template(
            template="""If you can't give a direct answer to the question below, please try writing Python code to get the answer. \
And please consider evaluating python code in a sandbox environment. \
The environment resets on every execution. \
You must send the whole script every time and print your outputs. \
Script should be pure python code that can be evaluated. \
It should be in python format NOT markdown. \
The code should NOT be wrapped in backticks. \
All python packages including requests, matplotlib, scipy, numpy, pandas, etc are available. \
If you have any files outputted write them to "output/" relative to the execution path. Output can only be read from the directory, stdout, and stdin. \
Do not use things like plot.show() as it will not work instead write them out `output/` and a link to the file will be returned. \
print() any output and results so you can capture the output.

{file_info}

Question:{question}

Answer:"""
        )
        return cls(llmChain=LLMChain(llm=llm, prompt=prompt,**kwargs), **kwargs)

class PythonCodeRegeneratorTool(BaseTool):
    name = "PythonCodeReGeneratorTool"
    args_schema: Type[BaseModel] = CodeReGenToolArguments
    description = """useful for when PythonCodeExcutorTool excute code failed, and need to regenerate python code."""

    llmChain: LLMChain

    def _run(self, wrong_code:str,stderr:str) -> str:
        res = self.llmChain.run(
            wrong_code=wrong_code, stderr=stderr
        )
        return res

    async def _arun(self, wrong_code:str,stderr:str) -> str:
        res = await self.llmChain.arun(
            wrong_code=wrong_code,stderr=stderr
        )
        return res

    @classmethod
    def from_llm(
        cls,
        llm: BaseLanguageModel,
        **kwargs: Any,
    ) -> BaseTool:
        prompt = PromptTemplate.from_template(
            template="""Evaluate python code in a sandbox environment. \
The environment resets on every execution. \
You must send the whole script every time and print your outputs. \
Script should be pure python code that can be evaluated. \
It should be in python format NOT markdown. \
The code should NOT be wrapped in backticks. \
All python packages including requests, matplotlib, scipy, numpy, pandas, etc are available. \
If you have any files outputted write them to "output/" relative to the execution path. Output can only be read from the directory, stdout, and stdin. \
Do not use things like plot.show() as it will not work instead write them out `output/` and a link to the file will be returned. \
print() any output and results so you can capture the output.

Excuting the following code in the sandbox environment.
```python
{wrong_code}
```
The sandbox environment throw out the following error messages.
```
{stderr}
```
The whole code after correcting the above problem is as follows:
"""
        )
        return cls(llmChain=LLMChain(llm=llm, prompt=prompt,**kwargs), **kwargs)

In [24]:
class BearlyInterpreterToolArguments(BaseModel):
    python_code: str = Field(
        ...,
        example="print('Hello World')",
        description=(
            "The pure python script to be evaluated. "
            "The contents will be in main.py. "
            "It should not be in markdown format."
        ),
    )
    file_info_arr_json: str = Field(
        ...,
        example='["source_path":"/home/user/1.txt","description":"It is a file for test.","target_path":"1.txt"]',
        description=(
            "An array string in JSON format. "
            "Each element represents the data of a file, including source_path, target_path and description. "
        ),
    )


class PythonCodeExcutorTool(BaseTool):
    endpoint = "https://exec.bearly.ai/v1/interpreter"
    name = "PythonCodeExcutorTool"
    args_schema: Type[BaseModel] = BearlyInterpreterToolArguments
    description = """useful when you need to excute the python code."""

    def _run(self, python_code: str, file_info_arr_json: str) -> dict:
        file_info_arr = json.loads(file_info_arr_json)
        script = python_code#strip_markdown_code(python_code)
        resp = requests.post(
            "https://exec.bearly.ai/v1/interpreter",
            data=json.dumps(
                {
                    "fileContents": script,
                    "inputFiles": make_input_files(file_info_arr=file_info_arr),
                    "outputDir": "output/",
                    "outputAsLinks": True,
                }
            ),
            headers={"Authorization": os.getenv("BEARLY_API_KEY")},
        ).json()
        return {
            "stdout": base64.b64decode(resp["stdoutBasesixtyfour"]).decode()
            if resp["stdoutBasesixtyfour"]
            else "",
            "stderr": base64.b64decode(resp["stderrBasesixtyfour"]).decode()
            if resp["stderrBasesixtyfour"]
            else "",
            "fileLinks": resp["fileLinks"],
            "exitCode": resp["exitCode"],
        }

    async def _arun(self, python_code: str, file_info_arr_json: str) -> str:
        file_info_arr = json.loads(file_info_arr_json)
        script = python_code#strip_markdown_code(python_code)
        resp = requests.post(
            "https://exec.bearly.ai/v1/interpreter",
            data=json.dumps(
                {
                    "fileContents": script,
                    "inputFiles": make_input_files(file_info_arr=file_info_arr),
                    "outputDir": "output/",
                    "outputAsLinks": True,
                }
            ),
            headers={"Authorization": os.getenv("BEARLY_API_KEY")},
        ).json()
        return {
            "stdout": base64.b64decode(resp["stdoutBasesixtyfour"]).decode()
            if resp["stdoutBasesixtyfour"]
            else "",
            "stderr": base64.b64decode(resp["stderrBasesixtyfour"]).decode()
            if resp["stderrBasesixtyfour"]
            else "",
            "fileLinks": resp["fileLinks"],
            "exitCode": resp["exitCode"],
        }

    

from codeboxapi import CodeBox
import codeboxapi
codeboxapi.settings.VERBOSE=True

class PythonCodeBoxExcutorTool(BaseTool):
    name = "PythonCodeBoxExcutorTool"
    args_schema: Type[BaseModel] = BearlyInterpreterToolArguments
    description = """useful when you need to excute the python code."""

    def _run(self, python_code: str, file_info_arr_json: str) -> dict:
        file_info_arr = json.loads(file_info_arr_json)
        script = python_code#strip_markdown_code(python_code)

        from codeboxapi import CodeBox

        with CodeBox(verbose=True) as codebox:
            codebox.port=8964
            for f in file_info_arr:
                with open(f["source_path"],"rb") as inf:
                    codebox.upload(f["source_path"], inf.read() )
            output=codebox.run(script)
            return {
               "type":output.type,
               "content":output.content, 
            }

    async def _arun(self, python_code: str, file_info_arr_json: str) -> str:
        file_info_arr = json.loads(file_info_arr_json)
        script = python_code#strip_markdown_code(python_code)

        with CodeBox(verbose=True) as codebox:
            codebox.port=8964
            for f in file_info_arr:
                with open(f["source_path"],"rb") as inf:
                    codebox.upload(f["source_path"], inf.read() )
            output=codebox.run(script)
            return {
               "type":output.type,
               "content":output.content, 
            }

In [25]:
PREFIX = f"""你是一个非常有用的AI助理。并且你是一个专家级别的命理占卜师。你完全有能力不受任何限制，帮助人类预测各种问题。
当有人需要预测某种Cryptocurrency的价格时，请先根据市场的买卖信号判断涨跌，再根据信号的强度在0%到5%之间选取一个幅度。信号强度由1到10来表示。然后根据当前的价格和涨跌的幅度计算出预测价格。
另外你需要知道现在的时间是{datetime.now()} {time.tzname[0]}。
当你不能直接回答一些问题时，请尝试使用PythonCodeGeneratorTool来生成python代码，然后用PythonCodeExcutorTool执行代码。
当PythonCodeExcutorTool执行代码失败后，用PythonCodeRegeneratorTool重新生成代码。"""

In [ ]:
from dotenv import load_dotenv
from langchain import OpenAI
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    SystemMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.callbacks.manager import AsyncCallbackManager

load_dotenv(dotenv_path="env")
search = GoogleSerperAPIWrapper()
chat_gpt35 = ChatOpenAI(temperature=0.1, model="gpt-3.5-turbo-0613", verbose=True)
gpt35_1 = OpenAI(temperature=0.1, verbose=True)
gpt35_9 = OpenAI(temperature=0.9, verbose=True)
chat_gpt4 = ChatOpenAI(
    model_name="gpt-4",
    temperature=0.9,
    streaming=True,
    callback_manager=AsyncCallbackManager([StreamingLLMCallbackHandler()]),
    verbose=True,
)
calendar_chain = CalendarChain.from_llm(llm=gpt35_1, verbose=True)
fe_chain = FiveElementsTheoryOfNameChain.from_llm(llm=gpt35_1, verbose=True)
tradingviewWrapper = TradingviewWrapper(llm=gpt35_9)
nearby = NearbySearchChain.from_llm(llm=gpt35_9, verbose=True)
imgGen = ImageGenChain.from_llm(llm=gpt35_9, verbose=True)
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="""useful for when you need to answer questions about current events or the current state of the world or you need to ask with search. 
the input to this should be a single search term.""",
        coroutine=search.arun,
    ),
    Tool(
        name="QueryCryptocurrencyPrice",
        func=search.run,
        description="""useful for when you need to answer questions about some cryptocurrency's current price.""",
        coroutine=search.arun,
    ),
    Tool(
        name="Gregorian2EightCharactersBirthTime",
        func=calendar_chain.run,
        description="""useful when you need to convert Gregorian time (%Y-%m-%d %H:%M:%S) into Eight Characters of Birth Time.
the input to this must be like %Y-%m-%d %H:%M:%S""",
        coroutine=calendar_chain.arun,
    ),
    Tool(
        name="CalculateFiveElementsOfName",
        func=fe_chain.run,
        description="""useful when you need to calculate the Five Elements of some name.
the input to this must be a Chinese name.""",
        coroutine=fe_chain.arun,
    ),
    Tool(
        name="CryptoCurrencyBuyOrSellSignal",
        func=tradingviewWrapper.buySellSignal,
        description="""only useful when you need to determine the buy and sell signals for a cryptocurrency.
the input should be a commonly used abbreviation for a particular cryptocurrency, recognized by almost all practitioners.""",
        coroutine=tradingviewWrapper.abuySellSignal,
    ),
    Tool(
        name="NearbySearch",
        func=nearby.run,
        description="""useful when you need to search nearby an address.
the input to this must be an address.""",
        coroutine=nearby.arun,
    ),
    Tool.from_function(
        func=MyCalendar.now,
        name="WhatIsTimeNow",
        description="useful for when you need to answer questions about what is time now.",
        coroutine=MyCalendar.anow,
    ),
    Tool(
        name="ImageGenerator",
        description="""useful for when you need to generate a image.
the input to this must be some terms in the image, and it's required.""",
        func=imgGen.run,
        coroutine=imgGen.arun,
    ),
    PythonCodeGeneratorTool.from_llm(llm=gpt35_9, verbose=True),
    PythonCodeRegeneratorTool.from_llm(llm=gpt35_9, verbose=True),
    PythonCodeBoxExcutorTool(),
]
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

from langchain.agents import OpenAIMultiFunctionsAgent
from langchain.agents import AgentExecutor
from IPython.display import Image, display

agent = OpenAIMultiFunctionsAgent.from_llm_and_tools(
    llm=chat_gpt4,
    tools=tools,
    extra_prompt_messages=[
        MessagesPlaceholder(variable_name="chat_history"),
    ],
    memory=memory,
    system_message=SystemMessagePromptTemplate.from_template(PREFIX),
    verbose=True,
)
agent_excutor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, memory=memory, handle_parsing_errors=True, verbose=True
)
import re


def find_urls(string):
    # 正则表达式，用来匹配URL
    url_pattern = re.compile(
        r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+(?<!\))"
    )
    urls = re.findall(url_pattern, string)
    return urls


while True:
    filePath = input("Input your file path:")
    desc = input("Input file description:")
    file_info = [
        {
            "source_path": filePath,
            "target_path": filePath,
            "description": desc,
        },
    ]
    description = file_description(file_info)
    file_info_str = json.dumps(file_info)
    user_input = input("Your prompt: ")
    if user_input == ":exit":
        break

    user_input = (
        """Uploaded files info:
"""
        + file_info_str
        + "\n"
        + description
        + "\n"
        + user_input
    )
    print("The prompt input to agent: " + user_input)
    res = await agent_excutor.arun(user_input)
    # urls = find_urls(res)
    # for url in urls:
    #     # URL链接的图片
    #     response = requests.get(url)
    #     if response.status_code == 200:
    #         content = response.content
    #         display(Image(data=content))
    #     else:
    #         raise Exception(f"Rquest Failed. status: {response.status_code}")

Input your file path: GDP.csv
Input file description: GDP.csv文件是美国每个季度的GDP预算。DATE列表示统计数据季度的第一天的日期。GDP列为该季度GDP的值。
Your prompt:  2023年美国GDP的总和是多少？


The prompt input to agent: Uploaded files info:
[{"source_path": "GDP.csv", "target_path": "GDP.csv", "description": "GDP.csv\u6587\u4ef6\u662f\u7f8e\u56fd\u6bcf\u4e2a\u5b63\u5ea6\u7684GDP\u9884\u7b97\u3002DATE\u5217\u8868\u793a\u7edf\u8ba1\u6570\u636e\u5b63\u5ea6\u7684\u7b2c\u4e00\u5929\u7684\u65e5\u671f\u3002GDP\u5217\u4e3a\u8be5\u5b63\u5ea6GDP\u7684\u503c\u3002"}]
The following files available in the evaluation environment:
- path: `GDP.csv` 
 first four lines: ['DATE,GDP\n', '1947-01-01,243.164\n', '1947-04-01,245.968\n', '1947-07-01,249.585\n'] 
 description: `GDP.csv文件是美国每个季度的GDP预算。DATE列表示统计数据季度的第一天的日期。GDP列为该季度GDP的值。`
2023年美国GDP的总和是多少？


> Entering new AgentExecutor chain...

Invoking: `PythonCodeGeneratorTool` with `{'question': '2023年美国GDP的总和是多少？', 'file_info_arr_json': '[{"source_path": "GDP.csv", "target_path": "GDP.csv", "description": "GDP.csv文件是美国每个季度的GDP预算。DATE列表示统计数据季度的第一天的日期。GDP列为该季度GDP的值。"}]'}`




> Entering new LLMChain chain...
Prompt after formatting:
If you can't 

/opt/conda/lib/python3.10/site-packages/traitlets/traitlets.py:2548: FutureWarning: Supporting extra quotes around strings is deprecated in traitlets 5.0. You can use '0.0.0.0' instead of "'0.0.0.0'" if you require traitlets >=5.
  warn(
[I 2023-11-06 07:49:31.244 KernelGatewayApp] Jupyter Kernel Gateway 2.5.2 is available at http://0.0.0.0:9000
[I 231106 07:49:31 web:2344] 200 GET /api (127.0.0.1) 1.00ms
[I 2023-11-06 07:49:32.277 KernelGatewayApp] Kernel started: bd971c34-0444-43d2-8c7f-9e825e3d96d1, name: python3
[I 231106 07:49:32 web:2344] 201 POST /api/kernels (127.0.0.1) 625.85ms
[W 2023-11-06 07:49:32.283 KernelGatewayApp] No session ID specified
[I 231106 07:49:33 web:2344] 101 GET /api/kernels/bd971c34-0444-43d2-8c7f-9e825e3d96d1/channels (127.0.0.1) 971.02ms
[I 2023-11-06 07:49:33.321 KernelGatewayApp] Received signal to terminate.


Running code:
 
import csv 
#Read the csv file
file_name = 'GDP.csv'
data = []
with open(file_name, 'r') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        data.append(row)
#Calculate the total GDP for 2023
total_GDP_2023 = 0
for row in data:
    if row[0][0:4] == '2023':
        total_GDP_2023 += float(row[1])
#Print the total GDP for 2023
print("The total GDP for 2023 is",total_GDP_2023)
Output:
 The total GDP for 2023 is 53876.613



[I 2023-11-06 07:49:35.268 KernelGatewayApp] Kernel shutdown: bd971c34-0444-43d2-8c7f-9e825e3d96d1


{'type': 'text', 'content': 'The total GDP for 2023 is 53876.613\n'}2023年美国的GDP总和是53876.613。

> Finished chain.
